# Introduction

The project aims to create a **Machine Learning** powered model which is capable of classifying the emotions of the text (in the form of short quotes/comments) with predefined labels of emotions.

Emotion detection, also known as sentiment analysis, is now one of the most attractive subfields of Machine Learning, especially Natural Language Processing (NLP) due to its wide applications in many aspects of modern life, such as providing emotional information to help people developing insights or making decisions. In this project, we specifically try to convey the main emotion of a comment sentence. Generally, we expect the model to be able to take the input sentence in the form of short comments and returns its major emotion(s), corresponding to one of the predefined labels.

Our approach to solving this problem is to split it into two halves:

  1.   Data Preprocessing
  2.   Model training and Result

For detail of each half, please go into the section


The dataset we use in this project is obtained from Kaggle: [EDFT Dataset](https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp)

Connect to Google Drive

# **Data Preprocessing**

## Importation & Data loading

Preparing necessary packages (may need to add more):

In [ ]:
import numpy as np  # Work with multi-dimenional data
import pandas as pd # Work with relational data
import matplotlib.pyplot as plt # Visualize data
import seaborn as sns # Visualize data base on matplotlib
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings("ignore")
# Because I hate seeing stupid warnings~

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
# Used for automatic feature extraction
from sklearn.pipeline import Pipeline
# Used for automating processes
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, cross_val_score
# Used for spliting test and tuning hyperparameter
from sklearn.model_selection import StratifiedShuffleSplit
# Used for k-fold validiator in multi-classficiation

from sklearn.neighbors import KNeighborsClassifier
# Defining kNN
from sklearn.tree import DecisionTreeClassifier
# Defining Decision Tree
from sklearn.ensemble import RandomForestClassifier
# Defining Random Forest
from sklearn.svm import SVC
# Defining Classifier SVM
from sklearn.naive_bayes import MultinomialNB, GaussianNB
# Defining Naive Bayes

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
# Used for calculating metric-related scores of the model performance, and plotting the confusion matrix

%matplotlib inline

Load data from files

In [ ]:
directory = 'data/dataset/'

# This method use to extract all the file in the input list
def extract_data(files):
  data_x_raw = list()
  data_y_raw = list()
  for file in files:
    with open(directory + file) as f:
      for line in f:
        line = line.strip('\n')
        x_raw, y_raw = line.split(sep=';')

        data_x_raw.append(x_raw)
        data_y_raw.append(y_raw)

  return data_x_raw, data_y_raw

In [ ]:
# Get all data

X_raw, y_raw = extract_data(['train.txt', 'val.txt', 'test.txt'])
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size = 0.2)

print("Traning data's size is:", len(X_train_raw))
print("Test data's size is:", len(X_test_raw))

## Inital feature Extraction


Removing the stopword

In [ ]:
file = open("data/stopwords/stop_words_english.txt", 'r')
stopword_list = file.read().split('\n')
file.close()

Vectorize the data:

1.   Bag Of Words model (https://en.wikipedia.org/wiki/Bag-of-words_model)
2.   BoW/TF-IDF model (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)



In [ ]:
# Scikit CountVectorizer:
# Documentation: (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

count_vector = CountVectorizer(stop_words=stopword_list, ngram_range=(1, 2))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow = count_vector.transform(X_train_raw)
X_test_bow = count_vector.transform(X_test_raw)
dictionary = count_vector.get_feature_names_out()

print("Shape of preprocessed training data X using BoW model is: ", X_train_bow.shape)

In [ ]:
# Scikit TF-IDF:
# Documentation: (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

tfidf_vector = TfidfVectorizer(stop_words=stopword_list, ngram_range=(1, 2))
tfidf_vector.fit(X_train_raw, X_test_raw)
X_train_tfidf = tfidf_vector.transform(X_train_raw)
X_test_tfidf = tfidf_vector.transform(X_test_raw)

print("Shape of preprocessed training data X using BoW/TF-IDF model is: ", X_train_tfidf.shape)

Cast output list to ndarray

In [ ]:
y_train = np.asarray(y_train_raw)
y_test = np.asarray(y_test_raw)

*Luckily we're quite done with the abbreviations!*

Another way: Using Word2Vec to capture the semantics
(Um, we'd talk about that later, ok?)


## Initial exploratory Data Analysis (EDA)

Average length of the comment:

In [ ]:
# Calculate average number of characters
sum_of_chars = sum([len(x) for x in X_train_raw])
mean_number_chars = sum_of_chars / len(X_train_raw)

# Calculate average number of words
sum_of_words = sum([len(x.split()) for x in X_train_raw])
mean_number_words = sum_of_words / len(X_train_raw)

# Print
print('The mean number of characters of each line is: %d' % mean_number_chars)
print('The mean number of words of each line is: %d' % mean_number_words)

Statistic of output labels:

In [ ]:
# Number of labels:
labels = np.unique(y_train)
print('Number of labels is: %d' % len(labels))
print('Labels: ' + ', '.join(labels))
print('\n----------------------------------------------------------------------------\n')

# Plot:
df = pd.DataFrame(
  dict(
    labels = labels,
    label_elements_count = [y_train_raw.count(label) for label in labels]
  )
)
df = df.sort_values('label_elements_count')

plt.title(label='Labels statistics')
plt.xlabel('Labels')
plt.ylabel('Count')

plt.bar('labels', 'label_elements_count', data=df)

plt.show()

Statistics of words with the highest frequency of each label:

In [ ]:
def show_highest_frequency_words():
  vect_arr = X_train_bow.toarray()
  vect_aggr = np.zeros(shape=(len(labels), vect_arr.shape[1]))

  for idx in range(len(vect_arr)):
    label_idx = int(np.where(labels==y_train[idx])[0][0])

    vect_aggr[label_idx] += vect_arr[idx]

  fig, axs = plt.subplots(3, 2)

  for label_idx in range(len(labels)):
    axs_x, axs_y = label_idx // 2, label_idx % 2

    plt_ref = axs[axs_x][axs_y]
    plt_ref.set_title(labels[label_idx].upper())

    x_axis, y_axis = list(), list()
    n_argmax = np.argpartition(vect_aggr[label_idx], -7)[-7:]
    n_argmax = n_argmax[np.argsort(vect_aggr[label_idx][n_argmax])]
    for i in n_argmax:
      x_axis.append(dictionary[i])
      y_axis.append(vect_aggr[label_idx][i])

    x_axis = np.asarray(x_axis)
    y_axis = np.asarray(y_axis)

    plt_ref.barh(x_axis, y_axis)

  # Adjust layout
  plt.tight_layout()

  # Show the plot
  plt.show()

show_highest_frequency_words()

## Data Fixing

This part will eliminate all the words that have bad influence to our data.
All the words will be stored in a file stored in file *filtered_words.txt*

* Words appears so many times but don't contribute much (*feel*, *feeling*,...).
* Words have no mean (*aa*,*ab*,...)
* Words appears only a few

In [ ]:
file = open("data/stopwords/filtered_words.txt", 'r')
filtered_words = file.read().split('\n')

new_stopwords = stopword_list + filtered_words

Create tokenize function with the help of nltk packages

In [ ]:
stemmer = PorterStemmer()

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in new_stopwords]
    stems = [stemmer.stem(token) for token in tokens]
    return stems

Fit the data again

In [ ]:
count_vector = CountVectorizer(stop_words=new_stopwords, tokenizer=tokenize, ngram_range=(1, 2))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow = count_vector.transform(X_train_raw)
X_test_bow = count_vector.transform(X_test_raw)
dictionary = count_vector.get_feature_names_out()

tfidf_vector = TfidfVectorizer(stop_words=new_stopwords, tokenizer=tokenize, ngram_range=(1, 2))
tfidf_vector.fit(X_train_raw, X_test_raw)
X_train_tfidf = tfidf_vector.transform(X_train_raw)
X_test_tfidf = tfidf_vector.transform(X_test_raw)

In [ ]:
len(dictionary)

Fixing results:

In [ ]:
show_highest_frequency_words()

## Variables summary




---


**X_train_raw**, **y_train_raw**: ***list***

Raw training data which is extracted directly from files.

---


**X_test**, **y_test**: ***list***

  Raw testing data which is extracted directly from files.

---


**stopword_list**: ***list***

 List of stop words used in BoW and BoW/TF-IDF models

---


**dictionary**: ***numpy.ndarray***

 Dictionary of words which is used to vectorized the data and number of occurences.

---


**X_train_bow**: ***scipy.sparse._csr.csr_matrix***

 Training data input which is vectorized by the BoW model.

---


**X_train_tfidf**: ***scipy.sparse._csr.csr_matrix***

 Training data input which is vectorized by the BoW/TFIDF model

---


**X_test_bow**: ***scipy.sparse._csr.csr_matrix***

 Testing data input which is vectorized by the BoW model.

---


**X_test_tfidf**: ***scipy.sparse._csr.csr_matrix***

 Testing data input which is vectorized by the BoW/TFIDF model

---


**y_train**: ***numpy.ndarray***

 Preprocessed traning data output


---

# **Model training and result**

## Preset functions

Current preset functions:

---

model_evaluate_full(model, X_train, X_test, y_train, y_test, include_training = False): Evaluate a model fully.
- model: Your model.
- X_train, X_test, y_train, y_test: The datasets used to evluate the model.
- include_training: Trigger True if you want to output the training metrics. On default, false.

---

draw_learning_curve(model, X_train, y_train, cv = 5, train_sizes = np.linspace(0.2, 1, 5), scoring = 'accuracy'): Draw the learning curve for a model.
- model: Your model.
- X_train, y_train: The datasets on which you want to draw the learning curve.
- cv: Number of cross-validation folds you want to commit. On default, 5.
- train_sizes: The threshold list for the learning curve to be evaluated. On default, [.2, .4, .6, .8, 1]
- scoring: Scoring metric used. On default, accuracy score is used.

model_evaluate_full

In [ ]:
def model_evaluate_full(model, X_train, X_test, y_train, y_test, include_training = False):

  model.fit(X_train, y_train)

  if include_training == True:
    y_pred = model.predict(X_train)
    print("Accuracy score on the train: {:.2f}".format(accuracy_score(y_pred, y_train)))
    print("F1 score on the train: {:.2f}".format(f1_score(y_pred, y_train, average = 'micro')))
    cm = confusion_matrix(y_pred, y_train)
    ConfusionMatrixDisplay(cm).plot()

  y_pred = model.predict(X_test)
  print("Accuracy score on the test: {:.2f}".format(accuracy_score(y_pred, y_test)))
  print("F1 score on the test: {:.2f}".format(f1_score(y_pred, y_test, average = 'micro')))
  cm = confusion_matrix(y_pred, y_test)
  ConfusionMatrixDisplay(cm).plot()

draw_learning_curve

In [ ]:
def draw_learning_curve(model, X_train, y_train, cv = 5, train_sizes = np.linspace(0.2, 1, 5), scoring = 'accuracy'):
  _, train_score, test_score = learning_curve(model, X_train_bow, y_train, n_jobs = -1, cv = cv,
                                              train_sizes = train_sizes, scoring = scoring)

  train_mean_score = np.mean(train_score, axis = 1)
  train_std_score = np.std(train_score, axis = 1)
  test_mean_score = np.mean(test_score, axis = 1)
  test_std_score = np.std(test_score, axis = 1)

  plt.fill_between(train_sizes, train_mean_score - train_std_score, train_mean_score + train_std_score, alpha = 0.1, color = 'g')
  plt.fill_between(train_sizes, test_mean_score - test_std_score, test_mean_score + test_std_score, alpha = 0.1, color = 'r')

  plt.plot(train_sizes, train_mean_score, color = 'g')
  plt.plot(train_sizes, test_mean_score, color = 'r')

## K-nearest neighbors (KNN)

**DEVELOPER'S NOTE: k-NN code will be the anchor code for implementing other models. Please refer to this code as a guide when you work on other classes of models. Please refer to SleepyHunter if you ever need any more preset functions, as well as when you find a bug. Regards.**

We define the model first

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 3)

Getting prediction on training set (without cross validation) then evaluate it!

In [ ]:
model_evaluate_full(knn_model, X_train_bow, X_test_bow, y_train, y_test, include_training = True)

Learning curve

In [ ]:
draw_learning_curve(knn_model, X_train_bow, y_train)

Actually I set k = 3 to beautify the record, I ran k >= 5 and it was terrible.
Now we draw the plot for a range of k-neighbors

In [ ]:
# Setting the hyperparameter range
K = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
accuracy_list = list()
valid_accuracy_list = list()

for k in K:
  knn_model = KNeighborsClassifier(n_neighbors = k)
  knn_model.fit(X_train_bow, y_train)

  data_pred_y = knn_model.predict(X_test_bow)
  data_valid_y = knn_model.predict(X_train_bow)

  accuracy_list.append([k, accuracy_score(y_test, data_pred_y)])
  valid_accuracy_list.append([k, accuracy_score(y_train, data_valid_y)])

accuracy_list = np.asarray(accuracy_list)
valid_accuracy_list = np.asarray(valid_accuracy_list)

plt.plot(accuracy_list[:, 0], accuracy_list[:, 1], label = "Testing accuracy", color = 'b')
plt.plot(valid_accuracy_list[:, 0], valid_accuracy_list[:, 1], label = "Training accuracy", color = 'r')
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Using GridSearchCV to make a total search

In [ ]:
dict_param = {'n_neighbors': np.arange(1, 11)}
best_knn_model = GridSearchCV(KNeighborsClassifier(), param_grid = dict_param, n_jobs = -1, cv = 10, scoring = 'accuracy')

best_knn_model.fit(X_train_bow, y_train)

print("Best parameters for k_NN:", best_knn_model.best_params_)
data_best_knn_pred_y = best_knn_model.predict(X_test_bow)
print("Accuracy of that model:", accuracy_score(data_best_knn_pred_y, y_test))


In [ ]:
text = "I feels shitty today"
print('fun day' in dictionary)
vect = count_vector.transform([text])
print(vect)
print(count_vector.inverse_transform(vect))

best_knn_model.predict(vect)

I'm going to do just literally the same thing for TfIdf dataset... later.

## Naive Bayes classification

# **Conclusion**